In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 🧱 Agent: Backtest the strategy with the model's predictions and calculate performance metrics for multiple tokens
def backtest_multiple_tokens_with_sentiment(all_tokens_data, all_models, initial_capital=10000):
    # Dictionary to store backtest results for each token
    all_backtest_results = {}

    # Loop through each token and backtest
    for token, combined_df in all_tokens_data.items():
        print(f"Backtesting strategy for {token}...")

        # Get the trained model for the current token
        model = all_models.get(token)

        if model is None:
            print(f"Model for {token} not found. Skipping backtest.")
            continue

        # Add predicted buy signals to the dataframe
        combined_df['predicted_buy_signal'] = model.predict(combined_df[['value_x', 'value_y', 'return', 'rolling_avg', 'value']])

        # Initialize portfolio variables
        capital = initial_capital  # Current capital
        positions = 0  # Number of tokens held
        portfolio_value = []  # List to track portfolio value over time
        buy_price = None  # Track the price at which tokens are bought

        # Simulate trades based on the predicted buy signals
        for index, row in combined_df.iterrows():
            # Buy signal: if the model predicts buy_signal == 1 and no tokens are held
            if row['predicted_buy_signal'] == 1 and positions == 0:
                positions = capital / row['value_y']  # Buy tokens at current price
                capital = 0  # All capital used for buying tokens
                buy_price = row['value_y']  # Track the price at which we bought

            # Sell signal: if the model predicts sell_signal == 1 and tokens are held
            elif row['sell_signal'] == 1 and positions > 0:
                capital = positions * row['value_y']  # Sell all tokens at current price
                positions = 0  # No tokens are held after selling
                buy_price = None  # Reset the buy price

            # Hold: when both buy and sell signals are false
            elif row['predicted_buy_signal'] == 0 and row['sell_signal'] == 0:
                # Holding the current position without action
                portfolio_value.append(capital + positions * row['value_y'])
                continue

            # Track portfolio value (capital + value of held positions)
            portfolio_value.append(capital + positions * row['value_y'])

        # Convert portfolio value to DataFrame for further analysis
        portfolio_df = pd.DataFrame(portfolio_value, columns=['portfolio_value'])
        portfolio_df['date'] = combined_df['datetime']  # Assuming 'datetime' column exists

        # Calculate total return
        total_return = (portfolio_df['portfolio_value'].iloc[-1] - initial_capital) / initial_capital * 100

        # Calculate drawdown (max drop from peak)
        portfolio_df['peak'] = portfolio_df['portfolio_value'].cummax()
        portfolio_df['drawdown'] = (portfolio_df['portfolio_value'] - portfolio_df['peak']) / portfolio_df['peak'] * 100
        max_drawdown = portfolio_df['drawdown'].min()

        # Calculate the Sharpe ratio
        portfolio_df['return'] = portfolio_df['portfolio_value'].pct_change() * 100  # Daily return
        average_daily_return = portfolio_df['return'].mean()
        risk_free_rate = 0  # Assuming risk-free rate is 0
        daily_volatility = portfolio_df['return'].std()
        sharpe_ratio = (average_daily_return - risk_free_rate) / daily_volatility

        # Store the backtest results for this token
        all_backtest_results[token] = {
            'portfolio_df': portfolio_df,
            'total_return': total_return,
            'max_drawdown': max_drawdown,
            'sharpe_ratio': sharpe_ratio
        }

        # Display the results
        print(f"Total Return for {token}: {total_return:.2f}%")
        print(f"Maximum Drawdown for {token}: {max_drawdown:.2f}%")
        print(f"Sharpe Ratio for {token}: {sharpe_ratio:.2f}")

        # Plot portfolio value over time
        portfolio_df.plot(x='date', y='portfolio_value', title=f'{token} Portfolio Value Over Time')
        plt.show()

    return all_backtest_results
